In [2]:
# # install necessary packages to access google drive.
# !apt-get install -y -qq software-properties-common python-software-properties module-init-tools
# !add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
# !apt-get update -qq 2>&1 > /dev/null
# !apt-get -y install -qq google-drive-ocamlfuse fuse

# from google.colab import auth
# auth.authenticate_user()
# from oauth2client.client import GoogleCredentials
# creds = GoogleCredentials.get_application_default()
# import getpass
# !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
# vcode = getpass.getpass()
# !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

# !mkdir -p drive
# !google-drive-ocamlfuse drive

E: Package 'python-software-properties' has no installation candidate

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [4]:
ls ./drive/5002final/Q6_img/

test_img/  train_img/


In [0]:
import numpy as np
import pandas as pd

import os
import cv2

In [0]:
data_root='./drive/5002final/Data_Q6/'
img_root = './drive/5002final/Q6_img/'

In [0]:
train_paths = []
test_paths = []

up_folder = os.path.join(data_root, 'train_video')
for root, folders, files in os.walk(up_folder):
    for file in files:
        train_paths.append(os.path.join(up_folder, file))
up_folder = os.path.join(data_root, 'test_video')
for root, folders, files in os.walk(up_folder):
    for file in files:
        test_paths.append(os.path.join(up_folder, file))

### Extract images from videos

In [0]:
# get this function from https://zhuanlan.zhihu.com/p/48419604
# and add some modification
def video2frames(video_number,
                 path_in='', 
                 path_out='', 
                 extract_number = 3, 
                 jpg_quality = 250):

    video_name = path_in.split('/')[-1].split('.')[0]
    cap = cv2.VideoCapture(path_in)  
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))  # frame number of video
    fps = cap.get(cv2.CAP_PROP_FPS)                    # frames per second
    dur = n_frames/fps                                 # duration of video

    success = True
    count = 0
    while success and count < extract_number:
        position = (count+1)/(extract_number+1)
        
        cap.set(cv2.CAP_PROP_POS_MSEC, int(dur*position*1000)) 
        success, image = cap.read()
        if success:
            print(f'{n}th video {video_name}, {count+1}th extracted')
            cv2.imwrite(os.path.join(path_out, f'{video_name}_{count}.jpg'), image, [int(cv2.IMWRITE_JPEG_QUALITY), jpg_quality])     # save frame as JPEG file
            count = count + 1
        else:
            throw(f'error occured in video {video_name}')


In [0]:
print('Train videos are extracting:')
n = 0
for p in train_paths:
    video2frames(n, path_in=p, path_out=os.path.join(data_root, 'train_img'))
    n += 1
    
print('Test videos are extracting:')
n = 0
for p in test_paths:
    video2frames(n, path_in=p, path_out=os.path.join(data_root, 'test_img'))
    n += 1

In [0]:
### Extract feature of images

In [12]:
from keras.preprocessing import image

from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

Using TensorFlow backend.


In [0]:
train_paths = []
up_folder = os.path.join(img_root, 'train_img/')
for root, folders, files in os.walk(up_folder):
    for file in files:
        train_paths.append(os.path.join(up_folder, file))
        

In [0]:
test_paths = []
up_folder = os.path.join(img_root, 'test_img')
for root, folders, files in os.walk(up_folder):
    for file in files:
        test_paths.append(os.path.join(up_folder, file))

In [0]:

def gen_video_features(fe_model, fe_input_prossesor, img_path_list):
    
    paths = img_path_list
    
    video_features = []
    video_names = []
    img_features = [[] for i in range(3)]
    count = 0
    for img_path in paths:        
        img_info = img_path.split('/')[-1].split('.')[0].split('_')
        img_num = int(img_info[-1])
        v_name = '_'.join(img_info[:-1])
        
        img_data = image.load_img(img_path, target_size=(224, 224))
        img_data = image.img_to_array(img_data)
        img_data = fe_input_prossesor(np.expand_dims(img_data, axis=0))
        features = fe_model.predict(img_data)
        features = np.array(features).flatten() # collapse feature into one-dim
        
        img_features[img_num].append(features)
        if img_num == 2:
            video_names.append(v_name)
            count += 1
            print(f'{count}th video processed')
    
    df_list = []
    for i in range(3):
        imgs = img_features[i]
        mn_features = np.array(imgs)
        
        feature_num = mn_features.shape[1]
        img_df = pd.DataFrame(imgs, columns=(np.array(range(feature_num))+feature_num*i))
        img_df['id'] = video_names

        df_list.append(img_df)
    
    return df_list

In [0]:
fe_model = MobileNet(weights='imagenet', include_top=False, pooling='avg')
fe_model.summary()

In [66]:

train_paths = sorted(train_paths)
train_list = gen_video_features(fe_model, preprocess_input, train_paths)


1th video processed
2th video processed
3th video processed
4th video processed
5th video processed
6th video processed
7th video processed
8th video processed
9th video processed
10th video processed
11th video processed
12th video processed
13th video processed
14th video processed
15th video processed
16th video processed
17th video processed
18th video processed
19th video processed
20th video processed
21th video processed
22th video processed
23th video processed
24th video processed
25th video processed
26th video processed
27th video processed
28th video processed
29th video processed
30th video processed
31th video processed
32th video processed
33th video processed
34th video processed
35th video processed
36th video processed
37th video processed
38th video processed
39th video processed
40th video processed
41th video processed
42th video processed
43th video processed
44th video processed
45th video processed
46th video processed
47th video processed
48th video processed
4

In [0]:
train_df = pd.merge(train_list[0], train_list[1], how='left', on='id')
train_df = pd.merge(train_df, train_list[2], how='left', on='id')
del train_list

In [0]:
train_df.to_csv(os.path.join(img_root, 'train_feature.csv'))

In [0]:
with open(os.path.join(data_root, 'tarin_tag.txt')) as f:
    content = f.readlines()
    video_names = [v.split(',')[0].split('.')[0] for v in content]
    tag = [int(t.split(',')[1].split('\\')[0]) for t in content]
    
label = pd.DataFrame({'id':video_names, 'label':tag})

In [0]:
train_df = pd.merge(train_df, label, how='left', on='id')
train_df = train_df.set_index('id')

In [109]:
test_paths[0]

'./drive/5002final/Q6_img/test_img/.DS_Store'

In [0]:
test_paths = sorted(test_paths)

In [115]:
test_list = gen_video_features(fe_model, preprocess_input, test_paths)

1th video processed
2th video processed
3th video processed
4th video processed
5th video processed
6th video processed
7th video processed
8th video processed
9th video processed
10th video processed
11th video processed
12th video processed
13th video processed
14th video processed
15th video processed
16th video processed
17th video processed
18th video processed
19th video processed
20th video processed
21th video processed
22th video processed
23th video processed
24th video processed
25th video processed
26th video processed
27th video processed
28th video processed
29th video processed
30th video processed
31th video processed
32th video processed
33th video processed
34th video processed
35th video processed
36th video processed
37th video processed
38th video processed
39th video processed
40th video processed
41th video processed
42th video processed
43th video processed
44th video processed
45th video processed
46th video processed
47th video processed
48th video processed
4

In [0]:
test_df = pd.merge(test_list[0], test_list[1], how='left', on='id')
test_df = pd.merge(test_df, test_list[2], how='left', on='id')
del test_list

In [0]:
test_df.to_csv(os.path.join(img_root, 'test_feature.csv'))

In [0]:
test_df = test_df.set_index('id')

In [0]:
train_df.to_csv(os.path.join(img_root, 'train_feature.csv'))

### Modeling

In [0]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [0]:
train_y = train_df[['label']]
train_x = train_df.drop('label', axis=1)
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.2)

In [0]:
xgb1  = XGBClassifier(booster='gbtree', n_estimators=600, learning_rate=0.1, max_depth=3)
# xgb2  = XGBClassifier(booster='gbtree', n_estimators=1000, learning_rate=0.1, max_depth=2)
# xgb3  = XGBClassifier(booster='gbtree', n_estimators=2000, learning_rate=0.1, max_depth=1)

In [85]:
xgb1.fit(x_train, y_train)
pred_y = xgb1.predict(x_test)
print(accuracy_score(y_test, pred_y))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8595641646489104


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


### Output result

In [0]:
from sklearn.externals import joblib

In [0]:
model = XGBClassifier(booster='gbtree', n_estimators=600, learning_rate=0.1, max_depth=3)
model.fit(train_x, train_y)

In [88]:
joblib.dump(xgb1, os.path.join(img_root, '5002Q6_model.joblib'))

['./drive/5002final/Q6_img/5002Q6_model.joblib']

In [119]:
test_df['label'] = xgb1.predict(test_df)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
result = test_df[['label']].reset_index()

In [122]:
result

,id,label
0,30018754,4
1,303746162,0
2,303771055,0
3,303783414,0
4,303785363,0
5,303835132,0
6,303860380,0
7,303866446,0
8,303922809,0
9,30393422,4


In [0]:
result['file_name'] = result['id'].map(lambda x: f'{x}.mp4')

In [0]:
result[['file_name', 'label']].to_csv(os.path.join(img_root, 'Q6_output.csv'), index=False)